In [ ]:
import numpy as np
import wandb

from wandb.keras import WandbCallback

from keras.layers import Activation, Dense, Dropout, Flatten
from keras.losses import MeanSquaredError as LossMSE
from keras.optimizers import Adam, SGD
from keras.models import Sequential

In [ ]:
training_data_path = 'data/'

In [ ]:
wandb.login()

In [ ]:
sweep_configuration = {
    "name": "arch8_hyperparameter_tuning",
    "method": "random",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dense_layer_1" : {"values" : [256, 512]},
        "dense_layer_2" : {"values" : [128, 256, 512]},
        "dense_layer_3" : {"values" : [128, 256, 512]},
        "dense_layer_4" : {"values" : [128, 256, 512]},
        "dense_layer_5" : {"values" : [128, 256]},
        "dropout_1": {"values": [0, 0.2, 0.4]},
        "dropout_2": {"values": [0, 0.2, 0.4]},
        "dropout_3": {"values": [0, 0.2, 0.4]},
        "dropout_4": {"values": [0, 0.2, 0.4]},
        "dropout_5": {"values": [0, 0.2, 0.4]},
        "learning_rate": {"distribution": "uniform", "min": 1e-4, "max": 1e-1},
        "batch_size": {"values": [16, 32, 64, 128, 264, 512]},
        "epochs": {"values": [10, 15, 20]},
        "optimizer": {"values": ["adam", "sgd"]},
        "activation_function": {"values": ["sigmoid", "relu"]}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="DP")

In [ ]:
try:
  train_data = np.load(training_data_path + 'train_data.npy')
  val_data = np.load(training_data_path + 'val_data.npy')
  train_target_ratings = np.load(training_data_path + 'train_target_ratings.npy')
  val_target_ratings = np.load(training_data_path + 'val_target_ratings.npy')

except OSError as e:
  print('\nERROR loading data')
  print(e)
  quit()

In [ ]:
def create_model(config, train_data_shape=None):
    model = Sequential()
    model.add(Dense(config['dense_layer_1'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_1']))

    model.add(Flatten())

    model.add(Dense(config['dense_layer_2'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_2']))

    model.add(Dense(config['dense_layer_3'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_3']))

    model.add(Dense(config['dense_layer_4'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_4']))

    model.add(Dense(config['dense_layer_5'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_5']))

    model.add(Dense(1))
    model.add(Activation('linear'))

    return model

In [ ]:
wandb.config = {
  "dense_layer_1": 512,
  "dense_layer_2": 512,
  "dense_layer_3": 256,
  "dense_layer_4": 128,
  "dense_layer_5": 128,
  "dropout_1": 0.2,
  "dropout_2": 0.2,
  "dropout_3": 0.2,
  "dropout_4": 0.2,
  "dropout_5": 0.2,
  "learning_rate": 0.001,
  "batch_size": 128,
  "epochs": 2,
  "optimizer": "adam",
  "activation_function": "sigmoid",
}

In [ ]:
def train():
    with wandb.init(
          project="DP",
          entity="xbarantomik",
          name="arch8_25m_added_imdb_context_max_abs_scaler_hyperparameter_tuning",
          group="hyperparameter_tuning"
        ) as run:
        config = wandb.config
        model = create_model(config, train_data_shape=train_data.shape[1])
        if config['optimizer'] == "sgd":
          model.compile(
            optimizer= SGD(
              learning_rate = config['learning_rate']
            ),
            loss=LossMSE(),
          )
        else:
          model.compile(
            optimizer= Adam(
              learning_rate = config['learning_rate']
            ),
            loss=LossMSE(),
          )
        model.fit(train_data, train_target_ratings, validation_data=(val_data, val_target_ratings), callbacks=[WandbCallback()], verbose=1)

In [ ]:
count = 20
wandb.agent(sweep_id, function=train, count=count)